In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np

# Parámetros del problema
L = 12e-6  # Longitud del canal en metros
a_l = 3e-9  # Radio en la punta en metros
a_r = 220e-9  # Radio en la base en metros
surface_charge_density = -1e18  # Densidad de carga en el canal (cargas/m^2)
KCl_concentration = 100e-3  # Concentración de KCl en mol/m^3 (100 mM)
D_K = 1.95e-9  # Coeficiente de difusión del K+ en m^2/s
D_Cl = 2.03e-9  # Coeficiente de difusión del Cl- en m^2/s
epsilon = 80 * 8.85e-12  # Permitividad del medio (agua)
V_applied = 0.1  # Voltaje aplicado en Voltios

# Crear el dominio del nanocanal
mesh = IntervalMesh(1000, 0, L)
V = FunctionSpace(mesh, 'P', 1)

# Funciones de prueba y de prueba
u = TrialFunction(V)
v = TestFunction(V)

# Definir la función de concentración inicial
c_K = Constant(KCl_concentration)
c_Cl = Constant(KCl_concentration)

# Definir la función de potencial inicial
phi = Function(V)

# Definir las ecuaciones de Poisson y Nernst-Planck
F = (epsilon * dot(grad(u), grad(v)) * dx - surface_charge_density * v * dx)

# Definir las condiciones de borde
bc = [DirichletBC(V, Constant(0), 'near(x[0], 0)'), DirichletBC(V, Constant(V_applied), 'near(x[0], L)')]

# Resolver el problema de Poisson
solve(lhs(F) == rhs(F), phi, bc)

# Calcular la densidad de corriente
J_K = -D_K * (grad(c_K) + c_K * grad(phi))
J_Cl = -D_Cl * (grad(c_Cl) + c_Cl * grad(phi))

# Calcular la corriente total
I = assemble((J_K + J_Cl) * dx)

# Graficar el potencial
plt.figure()
plot(phi, title='Potencial Eléctrico')
plt.xlabel('Longitud del canal (m)')
plt.ylabel('Potencial (V)')
plt.show()

# Graficar la densidad de corriente
J_total = project(J_K + J_Cl, V)
plt.figure()
plot(J_total, title='Densidad de Corriente')
plt.xlabel('Longitud del canal (m)')
plt.ylabel('Densidad de Corriente (A/m^2)')
plt.show()

# Imprimir el valor de la corriente total
print('Corriente total (A):', I)
